In [2]:
## PDE1
## 模型与剖分

import math
import numpy as np

from tool import PDE, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, H1Error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb, drawer_uh_u

n   = 4  #剖分次数
n   = n + 1 
Lam = np.array([1e0,1e4,1e8])
Mu  = Lam

H     = np.zeros(n)                                  #步长
P     = np.zeros((Lam.shape[0], n-1))                      #误差阶
L2P   = P.copy()
E     = np.zeros((Lam.shape[0],n), dtype=np.float64) #每个lambda(行)对应的误差(列)
L2E   = E.copy()

for i in range(len(Lam)):
    pde = PDE(Mu[i] + i * 1000, Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NC = cell.shape[0]
        #print("cr_NC= ", NC)
        # nn 特定情况下剖分次数
        nn  = math.log(NC/2, 4)
        NN = int(3 * 2 * 4**nn - (3 * 2 * 4**nn - 4 * 2**nn) / 2)
        cm = np.ones(NC, dtype=np.float64) / NC
     
        cr_node, cr_cell = get_cr_node_cell(node,cell)
        #print("cr_node= ", cr_node.shape[0])
        #print("cr_cell= ", cr_cell.shape[0])

        # 单元刚度矩阵和单元载荷向量 
        A1, A2 = get_stiff_and_div_matrix(cr_node, cr_cell, cm)
        bb = get_bb(pde, node, cell, cm)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, cr_node, cr_cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, cr_node, getIsBdNode)
        u = pde.solution(cr_node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = H1Error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        if j < n-1 :
            node, cell = uniform_refine(node, cell)
    uh_dir = "../../image/tmp/elaticity_uh_u/PDE1/uh_lam={}.png".format(Lam[i])
    u_dir = "../../image/tmp/elaticity_uh_u/PDE1/u_lam={}.png".format(Lam[i])
    drawer_uh_u(cr_node, uh, u, uh_dir, u_dir)

L2E = E * H

# 画图 得到误差阶
if n-1 > 1: 
    # 求误差阶 
    # 得到 P
    for i in range(len(Lam)):
        for j in range(n-1):
            #f = np.polyfit(np.log(H[j:j+1]), np.log(E[i][j:j+1]) ,1)
            #P[i][j] = f[0]
            P[i][j]  = np.log(E[i][j] / E[i][j+1]) / np.log(H[j] / H[j+1])
        
if n-1 > 1: 
    # 求误差阶 
    # 得到 P
    for i in range(len(Lam)):
        for j in range(n-1):
            #f = np.polyfit(np.log(H[j:j+1]), np.log(L2E[i][j:j+1]) ,1)
            #P[i][j] = f[0]
            L2P[i][j]  = np.log(L2E[i][j] / L2E[i][j+1]) / np.log(H[j] / H[j+1])

print("--------------------------H1--------------------------")
print_error(Lam, H, E)
print("--------------------------L2--------------------------")
print_error(Lam, H, L2E)
print("--------------------------P--------------------------")
if n-1 > 1:
    print_P(Lam, P)
print("--------------------------L2P--------------------------")
if n-1 > 1:
    print_P(Lam, L2P)

--------------------------H1--------------------------
---------------------Lam= 1.0---------------------

h=  1.0
e= 0.4102804175342446

h=  0.5
e= 0.14243719020779927

h=  0.25
e= 0.08382384186371196

h=  0.125
e= 0.04737414805480538

h=  0.0625
e= 0.02528276600610658


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.3954836377533817

h=  0.5
e= 0.13695582498223918

h=  0.25
e= 0.08048234975514776

h=  0.125
e= 0.04543017434466033

h=  0.0625
e= 0.024223779595908435


---------------------Lam= 100000000.0---------------------

h=  1.0
e= 0.4102771623077973

h=  0.5
e= 0.14243598273110478

h=  0.25
e= 0.0838231050604163

h=  0.125
e= 0.047373719419517415

h=  0.0625
e= 0.025282532500434746


--------------------------L2--------------------------
---------------------Lam= 1.0---------------------

h=  1.0
e= 0.4102804175342446

h=  0.5
e= 0.07121859510389963

h=  0.25
e= 0.02095596046592799

h=  0.125
e= 0.0059217685068506725

h=  0.0625
e= 0.0015801728753816612




In [1]:
## PDE2
## 模型与剖分

import math
import numpy as np

from tool import PDE2, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, H1Error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb, drawer_uh_u

n   = 4  #剖分次数
n   = n + 1 
Lam = np.array([1e0,1e4,1e8])
Mu  = Lam

H     = np.zeros(n)                                  #步长
P     = np.zeros((Lam.shape[0], n-1))                      #误差阶
L2P   = P.copy()
E     = np.zeros((Lam.shape[0],n), dtype=np.float64) #每个lambda(行)对应的误差(列)
L2E   = E.copy()

for i in range(len(Lam)):
    pde = PDE2(Mu[i] + i * 1000, Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NC = cell.shape[0]
        #print("cr_NC= ", NC)
        # nn 特定情况下剖分次数
        nn  = math.log(NC/2, 4)
        NN = int(3 * 2 * 4**nn - (3 * 2 * 4**nn - 4 * 2**nn) / 2)
        cm = np.ones(NC, dtype=np.float64) / NC
     
        cr_node, cr_cell = get_cr_node_cell(node,cell)
        #print("cr_node= ", cr_node.shape[0])
        #print("cr_cell= ", cr_cell.shape[0])

        # 单元刚度矩阵和单元载荷向量 
        A1, A2 = get_stiff_and_div_matrix(cr_node, cr_cell, cm)
        bb = get_bb(pde, node, cell, cm)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, cr_node, cr_cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, cr_node, getIsBdNode)
        u = pde.solution(cr_node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = H1Error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        if j < n-1 :
            node, cell = uniform_refine(node, cell)
    uh_dir = "../../image/tmp/elaticity_uh_u/PDE2/uh_lam={}.png".format(Lam[i])
    u_dir = "../../image/tmp/elaticity_uh_u/PDE2/u_lam={}.png".format(Lam[i])
    drawer_uh_u(cr_node, uh, u, uh_dir, u_dir)
L2E = E * H

# 画图 得到误差阶
if n-1 > 1: 
    # 求误差阶 
    # 得到 P
    for i in range(len(Lam)):
        for j in range(n-1):
            #f = np.polyfit(np.log(H[j:j+1]), np.log(E[i][j:j+1]) ,1)
            #P[i][j] = f[0]
            P[i][j]  = np.log(E[i][j] / E[i][j+1]) / np.log(H[j] / H[j+1])
        
if n-1 > 1: 
    # 求误差阶 
    # 得到 P
    for i in range(len(Lam)):
        for j in range(n-1):
            #f = np.polyfit(np.log(H[j:j+1]), np.log(L2E[i][j:j+1]) ,1)
            #P[i][j] = f[0]
            L2P[i][j]  = np.log(L2E[i][j] / L2E[i][j+1]) / np.log(H[j] / H[j+1])

print("--------------------------H1--------------------------")
print_error(Lam, H, E)
print("--------------------------L2--------------------------")
print_error(Lam, H, L2E)
print("--------------------------P--------------------------")
if n-1 > 1:
    print_P(Lam, P)
print("--------------------------L2P--------------------------")
if n-1 > 1:
    print_P(Lam, L2P)

--------------------------H1--------------------------
---------------------Lam= 1.0---------------------

h=  1.0
e= 0.13816708999878652

h=  0.5
e= 0.10384422008025787

h=  0.25
e= 0.04190134759642594

h=  0.125
e= 0.021359235974728382

h=  0.0625
e= 0.011133140131424525


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.13281022044434299

h=  0.5
e= 0.10010680004835941

h=  0.25
e= 0.04029172905261738

h=  0.125
e= 0.0204983463450698

h=  0.0625
e= 0.010673551631995081


---------------------Lam= 100000000.0---------------------

h=  1.0
e= 0.1381659115110543

h=  0.5
e= 0.10384339787568829

h=  0.25
e= 0.04190099306201377

h=  0.125
e= 0.021359046122610865

h=  0.0625
e= 0.011133038737335316


--------------------------L2--------------------------
---------------------Lam= 1.0---------------------

h=  1.0
e= 0.13816708999878652

h=  0.5
e= 0.051922110040128935

h=  0.25
e= 0.010475336899106485

h=  0.125
e= 0.0026699044968410478

h=  0.0625
e= 0.000695821258214

In [20]:
## PDE3
## 模型与剖分

import math
import numpy as np

from tool import PDE3, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, H1Error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb, drawer_uh_u

n   = 4  #剖分次数
n   = n + 1 
Lam = [1e0,1e4,1e8]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)

for i in range(len(Lam)):
    pde = PDE3(Mu[i] + i * 1000, Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NC = cell.shape[0]
        #print("cr_NC= ", NC)
        # nn 特定情况下剖分次数
        nn  = math.log(NC/2, 4)
        NN = int(3 * 2 * 4**nn - (3 * 2 * 4**nn - 4 * 2**nn) / 2)
        cm = np.ones(NC, dtype=np.float64) / NC
     
        cr_node, cr_cell = get_cr_node_cell(node,cell)
        #print("cr_node= ", cr_node.shape[0])
        #print("cr_cell= ", cr_cell.shape[0])

        # 单元刚度矩阵和单元载荷向量 
        A1, A2 = get_stiff_and_div_matrix(cr_node, cr_cell, cm)
        bb = get_bb(pde, node, cell, cm)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, cr_node, cr_cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, cr_node, getIsBdNode)
        u = pde.solution(cr_node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = H1Error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        if j < n-1 :
            node, cell = uniform_refine(node, cell)
    uh_dir = "../../image/tmp/elaticity_uh_u/PDE3/uh_lam={}.png".format(Lam[i])
    u_dir = "../../image/tmp/elaticity_uh_u/PDE3/u_lam={}.png".format(Lam[i])
    drawer_uh_u(cr_node, uh, u, uh_dir, u_dir)

if n-1 > 1: 
    """
    for i in range(len(Lam)):
        fig = plt.figure()
        plt.plot(np.log(H[1:]), np.log(E[i][1:]))
        plt.title("lam={}".format(Lam[i]))
        plt.xlabel("log(h)")
        plt.ylabel("log(e)")
        plt.savefig(fname="CRFem/elasticityCRFemLam_{}.png"
                    .format(Lam[i]))
        plt.close(fig)
    """ 
    for i in range(len(Lam)):
        f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]) ,1)
        P[i] = f[0]

#print("u= ", np.max(u))
#print("uh= ", np.max(uh))
print_error(Lam, H, E)
#print_error(Lam, H, E[isInterNode], E_rel[isInterNode])
if n-1 > 1:
    print_P(Lam, P)

---------------------Lam= 1.0---------------------

h=  1.0
e= 0.8170963137059355

h=  0.5
e= 0.34876125239808303

h=  0.25
e= 0.16691247112719212

h=  0.125
e= 0.08921846728849628

h=  0.0625
e= 0.047185016540124684


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.7867187156663246

h=  0.5
e= 0.33567802912663125

h=  0.25
e= 0.16012070953184152

h=  0.125
e= 0.08542417587534926

h=  0.0625
e= 0.04513468383827078


---------------------Lam= 100000000.0---------------------

h=  1.0
e= 0.8170896307680257

h=  0.5
e= 0.3487583718323451

h=  0.25
e= 0.16691097397331345

h=  0.125
e= 0.08921763065921373

h=  0.0625
e= 0.04718456442018082


---------------------误差阶---------------------
lam=  1.0
p=  0.9561194635126533

lam=  10000.0
p=  0.959075270179903

lam=  100000000.0
p=  0.9561200946855127

